# Clique simulation

This is a simplified simulation program that assumes different types of average behaviors for individuals/units. For example, what happens if a person switched between different groups of people every two weeks? What if this person wore a mask? What if this person quarantined for a week before meeting a new group?

The simulations can show how certain behaviors might have a much greater effect on suppressing the virus than others.


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import defaultdict
from enum import Enum

In [ ]:
#Event = namedtuple('Event', 'person, clique')

class Seir(Enum):
    S = 1
    E = 2
    I = 3
    R = 4


class Person:
    """Class representing a person who is part of one or more cliques."""
        
    def __init__(self, id, starting_clique, num_cliques, switching_frequency, initial_infection_rate,
                quarantine_days, is_asymptomatic):
        self.id = id
        self.clique_seq = []
        self.clique_seq.append(starting_clique)
        self.num_cliques = num_cliques
        self.switching_frequency = switching_frequency * num_cliques / (num_cliques - 1)
        self.quarantine_days = quarantine_days
        self.is_asymptomatic = is_asymptomatic
        if np.random.rand() < initial_infection_rate:
            self.state = Seir.I
            print('Person {} initially exposed'.format(id))
        else:
            self.state = Seir.S


    def simulate_behavior(self, time_steps):
        for i in range(time_steps):
            if np.random.rand() < self.switching_frequency:
                self.clique_seq.extend([-1] * self.quarantine_days)
                i += self.quarantine_days
                self.clique_seq.append(np.random.randint(self.num_cliques))
            else:
                self.clique_seq.append(self.clique_seq[-1])



In [ ]:
CLIQUE_COLORS = {
    'S': 'g',
    'E': 'y',
    'I': 'r',
    'R': 'k',
}

def display_clique(ax, id, clique, rows):
    if id >= 0:
        x_coord = int(id / rows)
        y_coord = id % rows
    else:
        raise ValueError('id {}'.format(id))
    x, y, c = [], [], []
    for i in range(len(clique)):
        x_offset = int(i / 3)
        y_offset = i % 3
        x.append(x_coord + x_offset / 10)
        y.append(y_coord + y_offset / 10)
        c.append(CLIQUE_COLORS[clique[i].state.name])
    ax.scatter(x, y, c=c)
        

In [ ]:
ROWS = 20
NUM_CLIQUES = 400
AVG_CLIQUE_SIZE = 5
NUM_PEOPLE = NUM_CLIQUES * AVG_CLIQUE_SIZE
TIME_STEPS = 300
INITIAL_INFECTION_RATE = 0.004 # Starting infection rate
# Masks block 99% of particles. Can we assume 99% reduction in exposures?
SOCIAL_DISTANCING_FACTOR = 0.05 # Multiply by this factor for transmission probability based on social distancing precautions
NUMBER_OF_RANDOM_EXPOSURES = 1 # Number of individuals encountered outside of clique on a given day
EXPOSED_TIME = 3.6 # SEIR model means first few days is likely non-infectious
INFECTED_TIME_ASYMPTOMATIC = 10 # days a person remains infectious if asymptomatic, will not self-quarantine
INFECTED_TIME_SYMPTOMATIC = 1.5 # days a person remains infectious if symptomatic, will self-quarantine
PROB_ASYMPTOMATIC = 0.5
QUARANTINE_DAYS = 14 # Wait this long before switching cliques
SWITCHING_FREQUENCY = 0#1/(28.0 - QUARANTINE_DAYS) # 1 / days between a person hopping to a new clique
CHANCE_OF_SPREAD = 0.25 # 1 / How often you meet up


# Monte Carlo Simulation

Randomly give people covid based on their risk profiles. Randomly simulate exposure time and recovery time.

In [ ]:
DEBUG = False

filtered_files = [file for file in os.listdir('../data/images/clique/') if file.startswith('pic_')]
for file in filtered_files:
    path_to_file = os.path.join('../data/images/clique/', file)
    os.remove(path_to_file)

people = []
for i in range(NUM_PEOPLE):
    is_asymptomatic = np.random.rand() < PROB_ASYMPTOMATIC
    people.append(Person(i, int(i * NUM_CLIQUES / NUM_PEOPLE), NUM_CLIQUES, SWITCHING_FREQUENCY,
                        INITIAL_INFECTION_RATE, QUARANTINE_DAYS, is_asymptomatic))

for i in range(NUM_PEOPLE):
    people[i].simulate_behavior(TIME_STEPS)
# When you mix, your risks are added with everyone in the group. Your risk to others outside the group are also increased.
daily_infected_hist = [0]
total_infected_hist = []
for t in range(TIME_STEPS):
    # Add each person to their new clique
    cliques = defaultdict(list)
    num_susceptible = 0
    num_exposed = 0
    num_infected = 0
    num_recovered = 0
    for i in range(NUM_PEOPLE):
        # Randomly progress through the SEIR model
        person = people[i]
        randval = np.random.rand()
        if person.state == Seir.E and randval < 1.0 / EXPOSED_TIME:
            person.state = Seir.I
            if DEBUG:
                print('{} is infected at time {}'.format(person.id, t))
        elif person.state == Seir.I:
            if person.is_asymptomatic and randval < 1.0 / INFECTED_TIME_ASYMPTOMATIC:
                person.state = Seir.R
            elif not person.is_asymptomatic and randval < 1.0 / INFECTED_TIME_SYMPTOMATIC:
                person.state = Seir.R
            if DEBUG:
                print('{} is recovered at time {}'.format(person.id, t))
        if people[i].clique_seq[t] >= 0:
            cliques[people[i].clique_seq[t]].append(people[i])
        if person.state == Seir.S:
            num_susceptible += 1
        if person.state == Seir.E:
            num_exposed += 1
        if person.state == Seir.I:
            num_infected += 1
        if person.state == Seir.R:
            num_recovered += 1
    # Aggregate risks for each person to their group
    max_clique_size = 0
    plt.figure(figsize=(15, 5))
    ax = plt.subplot(131)
    for i in range(NUM_CLIQUES):
        for p_rx in cliques[i]:
            clique_size = len(cliques[i])
            if len(cliques[i]) > max_clique_size:
                max_clique_size = clique_size
            if p_rx.state == Seir.S:
                for p_tx in cliques[i]:
                    if p_tx.state == Seir.I and np.random.rand() < CHANCE_OF_SPREAD:
                        p_rx.state = Seir.E
                        if DEBUG:
                            print('{} exposed {} at time {}'.format(p_tx.id, p_rx.id, t))
            if p_rx.clique_seq[t] >= 0 and p_rx.state == Seir.S:
                if np.random.rand() < SOCIAL_DISTANCING_FACTOR * NUMBER_OF_RANDOM_EXPOSURES * num_infected / NUM_PEOPLE:
                    p_rx.state = Seir.E
        display_clique(ax, i, cliques[i], ROWS)
    total_infected_hist.append(num_infected + num_recovered)
    if len(total_infected_hist) >= 2:
        daily_infected_hist.append(total_infected_hist[-1] - total_infected_hist[-2])
    ax.set_title('Day {}, Fraction that has had COVID {}'.format(t, (NUM_PEOPLE - num_susceptible) / NUM_PEOPLE))
    ax = plt.subplot(132)
    ax.plot(daily_infected_hist)
    ax.set_title('Daily cases')
    ax = plt.subplot(133)
    ax.plot(np.array(total_infected_hist)/NUM_PEOPLE)
    ax.set_title('Herd immunity')
    plt.savefig('../data/images/clique/pic_{:04d}.png'.format(t))
    plt.show()
    if DEBUG:
        print('Max clique size at time {}: {}'.format(t, max_clique_size))
print(sum([person.state != Seir.S for person in people]))